In [1]:
%load_ext autoreload
%autoreload 2
    
from datetime import datetime
import json

import argilla as rg
from datasets import load_dataset

client = rg.Argilla(api_url="http://localhost:6900/", api_key='a14427ea-9197-11ec-b909-0242ac120002')
# client = rg.Argilla(api_url="https://extralit-public-demo.hf.space/", api_key='a14427ea-9197-11ec-b909-0242ac120002')

/Users/jonny/micromamba/envs/extralit/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for dataset in client.datasets.list():
    print(dataset.name)
    # dataset.delete()

Table-Preprocessing
3-Integration-Validation
2-Data-Extraction
2-Data-Extraction
Review-Evaluations
Trial-Extraction-Compare
Trial-Extraction-Compare-Consensus
2-Data-Extractions
1-Master-Paper-List


# Load extraction dataset

In [3]:
dataset = client.datasets(
    name="2-Data-Extractions",
    workspace="itn-recalibration"
)
dataset

Dataset(id=UUID('3a7abf40-a6b7-4cf6-ac09-d89a8b33ac67') inserted_at=datetime.datetime(2024, 4, 4, 5, 23, 44, 562080) updated_at=datetime.datetime(2024, 11, 15, 0, 35, 14, 753190) name='2-Data-Extractions' status='ready' guidelines=None allow_extra_metadata=True distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('e9e4e699-a6f9-4482-b5dd-e45874bd87eb') last_activity_at=datetime.datetime(2024, 12, 1, 5, 55, 8, 469548))

## Update field

In [4]:
# Find the record with the specific metadata
records = dataset.records(query=rg.Query(filter=("metadata.reference", "==", "mosqueira2015pilot")))

# Update the record's extraction field
updated_records = []
for record in records:
    print(record.metadata)
    print(type(record.fields["extraction"]))
    record.fields["extraction"] = json.loads(record.fields["extraction"])
    print(type(record.fields["extraction"]))
    updated_records.append(record)

len(updated_records)

{'reference': 'mosqueira2015pilot', 'pmid': '25959771', 'doc_id': '276c32ef-26d2-40cb-b808-b764018cd2ea', 'type': 'Observation'}
<class 'str'>
<class 'dict'>
{'reference': 'mosqueira2015pilot', 'pmid': '25959771', 'doc_id': '276c32ef-26d2-40cb-b808-b764018cd2ea', 'type': 'ITNCondition'}
<class 'str'>
<class 'dict'>
{'reference': 'mosqueira2015pilot', 'pmid': '25959771', 'doc_id': '276c32ef-26d2-40cb-b808-b764018cd2ea', 'type': 'EntomologicalOutcome'}
<class 'str'>
<class 'dict'>
{'reference': 'mosqueira2015pilot', 'pmid': '25959771', 'doc_id': '276c32ef-26d2-40cb-b808-b764018cd2ea', 'type': 'ClinicalOutcome'}
<class 'str'>
<class 'dict'>


4

In [12]:
dataset.records.log(updated_records)

DatasetRecords: The provided batch size 256 was normalized. Using value 4.

Sending records...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.41s/batch]


DatasetRecords(Dataset(id=UUID('3a7abf40-a6b7-4cf6-ac09-d89a8b33ac67') inserted_at=datetime.datetime(2024, 4, 4, 5, 23, 44, 562080) updated_at=datetime.datetime(2024, 11, 15, 0, 35, 14, 753190) name='2-Data-Extractions' status='ready' guidelines=None allow_extra_metadata=True distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('e9e4e699-a6f9-4482-b5dd-e45874bd87eb') last_activity_at=datetime.datetime(2024, 11, 21, 18, 7, 47, 105497)))

In [10]:
json.loads(record.fields["extraction"])

{'reference': 'mosqueira2015pilot',
 'schema': {'fields': [{'name': 'observation_ref',
    'type': 'any',
    'extDtype': 'string'},
   {'name': 'itncondition_ref', 'type': 'any', 'extDtype': 'string'},
   {'name': 'N_people', 'type': 'integer'},
   {'name': 'Age_lower', 'type': 'number'},
   {'name': 'Age_upper', 'type': 'number'}],
  'primaryKey': ['observation_ref', 'itncondition_ref'],
  'pandas_version': '1.4.0'},
 'data': [{'observation_ref': 'S01',
   'itncondition_ref': 'N01',
   'N_people': 3903,
   'Age_lower': 0.5,
   'Age_upper': 14.0},
  {'observation_ref': 'S02',
   'itncondition_ref': 'N01',
   'N_people': 3903,
   'Age_lower': 0.5,
   'Age_upper': 14.0}],
 'validation': {'schema_type': 'dataframe',
  'version': '0.18.3',
  'columns': {'N_people': {'title': None,
    'description': 'Number of people in the study arm of the net in question',
    'dtype': 'int64',
    'nullable': True,
    'checks': None,
    'unique': False,
    'coerce': False,
    'required': True,
    

# Test dataset

## Custom Field

In [5]:
dataset = client.datasets(
    name="interactive_chat",
    # workspace="itn-recalibration"
)
dataset.delete()

/Users/jonny/Projects/extralit/argilla/src/argilla/client.py:354: UserWarning: Dataset with name 'interactive_chat' not found in workspace 'itn-recalibration'
  warnings.warn(f"Dataset with name {name!r} not found in workspace {workspace.name!r}")


AttributeError: 'NoneType' object has no attribute 'delete'

In [9]:
settings = rg.Settings(
    fields=[
        rg.TableField(name="chosen", required=False),
    ],
    questions=[
        rg.TableQuestion(
            "extraction", title="Correct the table", required=True
        ),
    ],
)

dataset = rg.Dataset(
    settings=settings,
    name=f"interactive_chat",
)

dataset.create()

Dataset(id=UUID('92b559e7-8eff-4d4c-85bf-817fd73570e4') inserted_at=datetime.datetime(2024, 12, 2, 21, 33, 33, 529345) updated_at=datetime.datetime(2024, 12, 2, 21, 33, 39, 111530) name='interactive_chat' status='ready' guidelines=None allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('e9e4e699-a6f9-4482-b5dd-e45874bd87eb') last_activity_at=datetime.datetime(2024, 12, 2, 21, 33, 39, 111530))

In [11]:
sample_table = record.fields['extraction']
sample_table

{'reference': 'mosqueira2015pilot',
 'schema': {'fields': [{'name': 'observation_ref',
    'type': 'any',
    'extDtype': 'string'},
   {'name': 'itncondition_ref', 'type': 'any', 'extDtype': 'string'},
   {'name': 'N_people', 'type': 'integer'},
   {'name': 'Age_lower', 'type': 'number'},
   {'name': 'Age_upper', 'type': 'number'}],
  'primaryKey': ['observation_ref', 'itncondition_ref'],
  'pandas_version': '1.4.0'},
 'data': [{'observation_ref': 'S01',
   'itncondition_ref': 'N01',
   'N_people': 3903,
   'Age_lower': 0.5,
   'Age_upper': 14.0},
  {'observation_ref': 'S02',
   'itncondition_ref': 'N01',
   'N_people': 3903,
   'Age_lower': 0.5,
   'Age_upper': 14.0}],
 'validation': {'schema_type': 'dataframe',
  'version': '0.18.3',
  'columns': {'N_people': {'title': None,
    'description': 'Number of people in the study arm of the net in question',
    'dtype': 'int64',
    'nullable': True,
    'checks': None,
    'unique': False,
    'coerce': False,
    'required': True,
    

In [8]:
dataset.records.log([
    {'chosen': sample_table} \
    for r in updated_records
])

DatasetRecords: The provided batch size 256 was normalized. Using value 4.

Sending records...: 100%|███████| 1/1 [00:02<00:00,  2.54s/batch]


DatasetRecords(Dataset(id=UUID('a64a827c-f962-417a-a771-ce53f61c0756') inserted_at=datetime.datetime(2024, 11, 29, 23, 9, 55, 104623) updated_at=datetime.datetime(2024, 11, 29, 23, 9, 58, 696913) name='interactive_chat' status='ready' guidelines=None allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('e9e4e699-a6f9-4482-b5dd-e45874bd87eb') last_activity_at=datetime.datetime(2024, 11, 29, 23, 9, 58, 696913)))

In [9]:
# ds = load_dataset("argilla/Capybara-Preferences", split="train[:100]")
# ds = ds.map(lambda x: {"messages": x["chosen"]})
# dataset.records.log(ds)